In [1]:
from keras.preprocessing.image import ImageDataGenerator


from keras.utils import np_utils
from keras.models import Sequential
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

In [8]:
path = r"C:\Users\oym91\OneDrive\Desktop\graph_all\\"
os.listdir(path+'train')

import shutil
trainFileList = os.listdir(path+'train')
testFileList = os.listdir(path+'test')

In [9]:
trainFileList = [x for x in trainFileList if 'png' in x]
testFileList = [x for x in testFileList if 'png' in x]

In [10]:
word = {'balance' : 0, 'decrease' : 1, 'increase' : 2, 'shortcycle' : 3}

In [11]:
# 폴더 만들어주기 train
currentPath = path + 'train'  # train 파일이 있는 현재 경로

print(word.keys()) # balance, increase, decrease, shortcycle

# 카테고리화 된 폴더가 없으면 폴더 생성
try:
    for key in word.keys(): 
        os.makedirs(currentPath + f'\\{key}')
# 이미 존재하면 패스
except:
    pass

# jpg 파일 옮기기
for file in trainFileList: # Train data들 각각
    label = file.split('_')[0] 
    # 예시 balance_01.jpg
    # label = 'balance'
    targetPath = currentPath + f'\\{label}' # 목표 위치인 aa폴더에 넣기
    try :
        shutil.move(currentPath+f'\\{file}', targetPath+f'\\{file}')
    except:
        pass 

dict_keys(['balance', 'decrease', 'increase', 'shortcycle'])


In [12]:
# 폴더 만들어주기 test
currentPath = path + 'test'
print(word.keys())
try:
    for key in word.keys(): # ga da ...
        os.makedirs(currentPath + f'\\{key}')
except:
    pass

for file in testFileList:
    label = file.split('_')[0]
    targetPath = currentPath + f'\\{label}'
    try :
        shutil.move(currentPath+f'\\{file}', targetPath+f'\\{file}')
    except:
        pass

dict_keys(['balance', 'decrease', 'increase', 'shortcycle'])


In [15]:
trainDataGen = ImageDataGenerator(rescale=1./255,
                                 rotation_range = 30,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=False,
                                 vertical_flip=False,
                                 fill_mode='nearest'
                                 )

trainGenSet = trainDataGen.flow_from_directory(
    path + 'train',
    batch_size=32,
    target_size=(288,864),
    class_mode='categorical'
)

Found 79 images belonging to 4 classes.


In [16]:
testDataGen = ImageDataGenerator(rescale=1./255)

testGenSet = testDataGen.flow_from_directory(
    path + 'test',
    target_size=(288,864),
    batch_size=32,
    class_mode='categorical'
)

Found 79 images belonging to 4 classes.


In [17]:
# 모델 구성
# 기존 모델 형식 그대로 가져왔음
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), padding='same', input_shape=(288,864,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 288, 864, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 144, 432, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 144, 432, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 144, 432, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 72, 216, 128)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 72, 216, 128)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 216, 256)      2

In [27]:
#모델 학습 설정
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [28]:

old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

# fig_generator
model.fit_generator(
    trainGenSet,
    steps_per_epoch=20,
    epochs=200,
    validation_data=testGenSet,
    validation_steps=10,
)

AttributeError: module 'tensorflow' has no attribute 'logging'

In [25]:
scores = model.evaluate_generator(testGenSet)
print(scores)

Instructions for updating:
Please use Model.evaluate, which supports generators.


InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[32,5] labels_size=[32,4]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at <ipython-input-25-316a1e8ed66d>:1) ]] [Op:__inference_test_function_1269]

Function call stack:
test_function
